In [40]:
#%pip install pandas pyarrow
import pandas as pd

# Load the Parquet file
articles = pd.read_parquet('articles.parquet', engine='pyarrow')
behaviors = pd.read_parquet('train/behaviors.parquet', engine='pyarrow')
history = pd.read_parquet('train/history.parquet', engine='pyarrow')

val_behaviors = pd.read_parquet('validation/behaviors.parquet', engine='pyarrow')
val_history = pd.read_parquet('validation/history.parquet', engine='pyarrow')

# bert = pd.read_parquet('bert_base_multilingual_cased.parquet', engine='pyarrow')
# display = bert['google-bert/bert-base-multilingual-cased']

# Test wikipedia entity embeddings

In [41]:
articles['ner_clusters']

0                                                       []
1                                    [Harry, James Hewitt]
2                                                       []
3                                                       []
4                                                       []
                               ...                        
11772    [Hawaiis, Kilauea, Kilauea, Reuters, US Geolog...
11773    [Alexandra Bøje, All England, Amalie Magelund,...
11774    [Cooper, Englemageren, Julie R, Julie R. Ølgaa...
11775    [Dnepr, Kherson, Nova Kakhovka, Nova Kakhovka,...
11776    [Adam, Bente, Birthe, Fyn, Fyns Politi, Hernin...
Name: ner_clusters, Length: 11777, dtype: object

In [42]:
articles['topics']

0        [Kriminalitet, Kendt, Sport, Katastrofe, Mindr...
1        [Kriminalitet, Kendt, Underholdning, Personfar...
2                     [Kriminalitet, Transportmiddel, Bil]
3        [Erhverv, Privat virksomhed, Livsstil, Familie...
4                [Kriminalitet, Personfarlig kriminalitet]
                               ...                        
11772                [Katastrofe, Vejr, Større katastrofe]
11773    [Kendt, Begivenhed, Sport, Ketcher- og batspor...
11774    [Kendt, Livsstil, Familieliv, Underholdning, F...
11775    [International politik, Katastrofe, Større kat...
11776                            [Kriminalitet, Bedrageri]
Name: topics, Length: 11777, dtype: object

In [43]:
with open("../../../../glove.840B.300d.txt", "r", encoding="utf8") as f:
    lines = f.readlines()
    print(lines[0]) 

, -0.082752 0.67204 -0.14987 -0.064983 0.056491 0.40228 0.0027747 -0.3311 -0.30691 2.0817 0.031819 0.013643 0.30265 0.0071297 -0.5819 -0.2774 -0.062254 1.1451 -0.24232 0.1235 -0.12243 0.33152 -0.006162 -0.30541 -0.13057 -0.054601 0.037083 -0.070552 0.5893 -0.30385 0.2898 -0.14653 -0.27052 0.37161 0.32031 -0.29125 0.0052483 -0.13212 -0.052736 0.087349 -0.26668 -0.16897 0.015162 -0.0083746 -0.14871 0.23413 -0.20719 -0.091386 0.40075 -0.17223 0.18145 0.37586 -0.28682 0.37289 -0.16185 0.18008 0.3032 -0.13216 0.18352 0.095759 0.094916 0.008289 0.11761 0.34046 0.03677 -0.29077 0.058303 -0.027814 0.082941 0.1862 -0.031494 0.27985 -0.074412 -0.13762 -0.21866 0.18138 0.040855 -0.113 0.24107 0.3657 -0.27525 -0.05684 0.34872 0.011884 0.14517 -0.71395 0.48497 0.14807 0.62287 0.20599 0.58379 -0.13438 0.40207 0.18311 0.28021 -0.42349 -0.25626 0.17715 -0.54095 0.16596 -0.036058 0.08499 -0.64989 0.075549 -0.28831 0.40626 -0.2802 0.094062 0.32406 0.28437 -0.26341 0.11553 0.071918 -0.47215 -0.18366 -0.3

## Topics

In [44]:
topics = []
nr_articles_no_topics = 0
for topicline in articles["topics"]:
    if len(topics) == 0:
        nr_articles_no_topics += 1
    for topic in topicline:
        topics.append(topic)
unique_topics = set(topics)
print(f"Number of unique topics: {len(unique_topics)}")
print(f"Percentage of articles without topics: {(nr_articles_no_topics / len(articles)) * 100}%")

Number of unique topics: 78
Percentage of articles without topics: 0.008491126772522714%


In [45]:
found = 0
found_topics = []
for line in lines[1:]:
    topic = line.split()[0]
    # print(topic)
    if topic in unique_topics:
        found += 1
        found_topics.append(topic)
print(f"Percentage of topics found in the wikipedia embeddings: {(found/len(unique_topics)  * 100)}%")   

Percentage of topics found in the wikipedia embeddings: 16.666666666666664%


In [46]:
# write to file
with open("../entity/topics2id.txt", "w", encoding="utf8") as f:
    for idx,topic in enumerate(set(found_topics)):
        f.write(str(idx+1) +"\t" + topic + "\n")
print(idx)

12


In [47]:
# test file
with open('../entity/topics2id.txt', encoding="utf8") as f:
    lines = f.readlines()

Topics2Index = {}
Index2Topics = {}
print(len(lines))
for line in lines:
    index, word = line.strip('\n').split('\t')
    if word in Topics2Index or index in Index2Topics:
        raise Exception("Duplicate topic")
    Topics2Index[word] = int(index)
    Index2Topics[int(index)] = word
len(Topics2Index)

13


13

In [48]:
doc2topicsid = {}
for idx, article in articles.iterrows():
    id = article["article_id"]
    doc_topics = article["topics"]
    for doc_topic in doc_topics:
        if doc_topic not in Topics2Index:
            continue
        if id not in doc2topicsid:
            doc2topicsid[id] = []
        doc2topicsid[id].append(Topics2Index[doc_topic])
            
len(doc2topicsid)

5877

In [49]:
# write
with open("../entity/doc2topicsid.txt", "w", encoding="utf8") as f:
    for doc_id, topics in doc2topicsid.items():
        f.write(str(doc_id) + "\t" + " ".join([str(topic) for topic in topics]) + "\n")

# test
test_dict = {}
with open('../entity/doc2topicsid.txt', encoding="utf8") as f:
    lines = f.readlines()
    for line in lines:
        doc_id, topics = line.strip('\n').split('\t')
        topics = topics.split()
        test_dict[doc_id] = [int(topic) for topic in topics]
test_dict   

{'3037230': [8],
 '3057622': [7],
 '3073151': [6],
 '3196611': [6],
 '3251664': [8],
 '3971783': [1],
 '3977720': [9],
 '3987418': [8],
 '3997229': [13],
 '3998680': [8],
 '4000823': [8],
 '4003926': [8],
 '4007811': [9],
 '4021392': [6],
 '4024449': [7],
 '4035605': [8],
 '4043121': [7],
 '4044042': [13],
 '4045439': [13],
 '4048845': [9],
 '4050800': [9],
 '4066000': [1, 9],
 '4067703': [8],
 '4069792': [9],
 '4081011': [11],
 '4081857': [8],
 '4085973': [8],
 '4086433': [8],
 '4096785': [8],
 '4106368': [11],
 '4116752': [4],
 '4117997': [9],
 '4152423': [9],
 '4160452': [8, 13],
 '4160891': [9],
 '4163017': [13, 6],
 '4171506': [8],
 '4174342': [13],
 '4179715': [7],
 '4212112': [11],
 '4212220': [11, 6, 2],
 '4219741': [1, 9],
 '4219799': [6, 2],
 '4225213': [9],
 '4233674': [9],
 '4234038': [8],
 '4241055': [9],
 '4251878': [9],
 '4257110': [9],
 '4280152': [11],
 '4283352': [7, 13],
 '4288610': [13, 11],
 '4297775': [9],
 '4313550': [11, 12],
 '4327174': [6],
 '4340048': [8, 5],

## Entities

In [50]:
entities = []
nr_articles_no_entities = 0
for ner_clusters in articles["ner_clusters"]:
    if len(ner_clusters) == 0:
        nr_articles_no_entities += 1
    for entity in ner_clusters:
        entities.append(entity)
unique_entities = set(entities)
print(f"Number of unique entities: {len(unique_entities)}")
print(f"Percentage of articles without entities: {(nr_articles_no_entities / len(articles)) * 100}%")

Number of unique entities: 33078
Percentage of articles without entities: 14.32453086524582%


In [51]:
found = 0
found_embeddings = []
for line in lines[1:]:
    entity = line.split()[0]
    if entity in unique_entities:
        found += 1
        found_embeddings.append(entity)
print(f"Percentage of entities found in the wikipedia embeddings: {(found/len(unique_entities)  * 100)}%")    

Percentage of entities found in the wikipedia embeddings: 0.0%


In [52]:
# write to file
with open("../entity/entities2id.txt", "w", encoding="utf8") as f:
    for idx,entity in enumerate(set(found_embeddings)):
        f.write(str(idx+1) +"\t" + entity + "\n")
print(idx)

11776


In [53]:
# test file
with open('../entity/entities2id.txt', encoding="utf8") as f:
    lines = f.readlines()

Entity2Index = {}
Index2Entity = {}
print(len(lines))
for line in lines:
    index, word = line.strip('\n').split('\t')
    if word in Entity2Index or index in Index2Entity:
        raise Exception("Duplicate entity")
    Entity2Index[word] = int(index)
    Index2Entity[int(index)] = word
len(Entity2Index)

0


0

In [54]:
Entity2Index

{}

In [55]:
len(unique_entities)

33078

In [56]:
doc2entitiesid = {}
for idx, article in articles.iterrows():
    id = article["article_id"]
    doc_entities = article["ner_clusters"]
    for doc_entity in doc_entities:
        if doc_entity not in Entity2Index:
            continue
        if id not in doc2entitiesid:
            doc2entitiesid[id] = []
        doc2entitiesid[id].append(Entity2Index[doc_entity])
            
len(doc2entitiesid)

0

In [57]:
# write
with open("../entity/doc2entitiesid.txt", "w", encoding="utf8") as f:
    for doc_id, entities in doc2entitiesid.items():
        f.write(str(doc_id) + "\t" + " ".join([str(entity) for entity in entities]) + "\n")

# test
test_dict = {}
with open('../entity/doc2entitiesid.txt', encoding="utf8") as f:
    lines = f.readlines()
    for line in lines:
        doc_id, entities = line.strip('\n').split('\t')
        entities = entities.split()
        test_dict[doc_id] = [int(entity) for entity in entities]
test_dict         

{}

## Create mergeimps.tsv (popularity)
The "mergeimps.tsv'' file needs to store the #click and #impression of each news with time information for building popularity signals. Specifically, we first need to divide the time into several buckets in units of one hour. Thus, since the MIND dataset contains user data in 6 weeks, the time periods can be divided into 2476 buckets. Then, we count the #impressions and #clicks of a news in each bucket and store the results in each line of ``mergeimps.tsv'', where the format is :

[News_ID] TAB [Bucket_ID] TAB #Cliks TAB #Impressions.
 
In MIND, since we did not obtain the click information for test data, which can substantially affect the estimation of popularity signals. Thus, I advise that you can use the validation dataset of MIND for evaluation.

In [58]:
# anchor = trans2tsp('05/08/2023 11:59:59 PM')
# def parse_time_bucket(date):
#     tsp = trans2tsp(date)
#     tsp = tsp - anchor
#     tsp = tsp//(600 * 2.2)
#     return tsp

def create_mergeimps(behaviors):
    # Assuming 'impression_time' is already in datetime format, if not convert it
    behaviors['impression_time'] = pd.to_datetime(behaviors['impression_time'])

    # Set the start date of your data collection period
    start_date = pd.to_datetime('2023-05-18')

    # Calculate the hour buckets since the start of the period
    behaviors['bucket_id'] = ((behaviors['impression_time'] - start_date).dt.total_seconds() // (1200)).astype(int)

    # Check if 'article_ids_inview' is already in the desired format
    if not behaviors['article_ids_inview'].str.contains('N').any():
        # For each line in the 'article_ids_inview' column, add 'N' upfront
        # Add 'N' before each 'article_ids_inview', replace each space with ' N' and remove square brackets
        behaviors['article_ids_inview'] = behaviors['article_ids_inview'].apply(lambda x: 'N' + str(x))
        behaviors['article_ids_inview'] = behaviors['article_ids_inview'].str.replace(' ', ' N').str.replace('[', '').str.replace(']', '').str.replace('\n', '')

    # Same for article_ids_clicked
    if not behaviors['article_ids_clicked'].str.contains('N').any():
        behaviors['article_ids_clicked'] = behaviors['article_ids_clicked'].apply(lambda x: 'N' + str(x) )
        behaviors['article_ids_clicked'] = behaviors['article_ids_clicked'].str.replace(' ', ' N').str.replace('[', '').str.replace(']', '')

    double_dict = {}

    for i, row in behaviors.iterrows():

        for article_id in row["article_ids_inview"].split(' '):
            if str(article_id) + "-" + str(row["bucket_id"]) not in double_dict:
                double_dict[str(article_id) + "-" + str(row["bucket_id"])] = {"article_id": article_id, "bucket_id": row["bucket_id"], "clicks": 0, "impressions": 0}
            double_dict[str(article_id) + "-" + str(row["bucket_id"])]["impressions"] += 1
                
        for article_id in row["article_ids_clicked"].split(' '):
            if str(article_id) + "-" + str(row["bucket_id"]) not in double_dict:
                double_dict[str(article_id) + "-" + str(row["bucket_id"])] = {"article_id": article_id, "bucket_id": row["bucket_id"], "clicks": 0, "impressions": 0}
            double_dict[str(article_id) + "-" + str(row["bucket_id"])]["clicks"] += 1
            
    mergeimps = pd.DataFrame(double_dict.values())

    return mergeimps

mergeimps_train = create_mergeimps(behaviors)
mergeimps_val = create_mergeimps(val_behaviors)



In [59]:
mergeimps =pd.concat([mergeimps_train, mergeimps_val])

In [61]:
# mergeimps.to_excel("mergeimps.xlsx", index=False)
mergeimps.to_csv("../popularity/mergeimps.tsv", sep="\t", index=False, header=False)

In [62]:
# tests
test = mergeimps.groupby("article_id").agg({"clicks": "sum", "impressions": "sum", "bucket_id" :["nunique", "count"]}).reset_index()
test

article_id clicks impressions bucket_id      
                   sum         sum   nunique count
0      N4721905      1           9         6     6
1      N4918926      0           5         3     3
2      N5345489      0           1         1     1
3      N5372434      0           1         1     1
4      N5824120      0           8         8     8
...         ...    ...         ...       ...   ...
3724   N9791130      2          37         3     3
3725   N9791134      0           3         1     1
3726   N9791141      1          29         2     2
3727   N9791157      2          10         1     1
3728   N9791165      3          13         1     1

[3729 rows x 5 columns]

## Create V21UrlDocs22_title_json.tsv (necessary?)
For implementing our method based on MIND, the "V21UrlDocs22_title_json.tsv'' needs to store all news in the training/val/test set. Each line stores the information of a single news article, where format is based on 

[News_ID] TAB [Category] TAB [Subcategory] TAB [Title] TAB [Url] TAB [Abstract] TAB [Body].
 
You can modify my codes to avoid the load of some unnecessary news data (e.g., Url, Abstract, and Body)

## Create train/val/test.tsv

In [63]:
def process_ids(row):
    clicked_ids = set(row['article_ids_clicked'].split())
    inview_ids = row['article_ids_inview'].split()
    processed_ids = [id for id in inview_ids if id not in clicked_ids]
    return ' '.join(processed_ids)

def write_data(behaviors, history, max_history, trainset):
    # Convert 'impression_time' to string
    behaviors['impression_time'] = behaviors['impression_time'].astype(str)

    # Check if 'impression_time' is already in the desired format
    if not behaviors['impression_time'].str.contains('/').any():
        # Convert 'impression_time' to datetime
        behaviors['impression_time'] = pd.to_datetime(behaviors['impression_time'])

        # Format 'impression_time' in the desired format
        behaviors['impression_time'] = behaviors['impression_time'].dt.strftime('%m/%d/%Y %I:%M:%S %p')

    # Add 'U' before every 'user_id' if it doesn't already start with 'U'
    behaviors['user_id'] = behaviors['user_id'].apply(lambda x: 'U' + str(x) if not str(x).startswith('U') else str(x))

    # Convert 'article_ids_inview' to string
    behaviors['article_ids_inview'] = behaviors['article_ids_inview'].astype(str)

    # Check if 'article_ids_inview' is already in the desired format
    if not behaviors['article_ids_inview'].str.contains('N').any():
        # For each line in the 'article_ids_inview' column, add 'N' upfront
        # Add 'N' before each 'article_ids_inview', replace each space with ' N' and remove square brackets
        behaviors['article_ids_inview'] = behaviors['article_ids_inview'].apply(lambda x: 'N' + str(x))
        behaviors['article_ids_inview'] = behaviors['article_ids_inview'].str.replace(' ', ' N').str.replace('[', '').str.replace(']', '').str.replace('\n', '')

    # Same for article_ids_clicked
    if not behaviors['article_ids_clicked'].str.contains('N').any():
        behaviors['article_ids_clicked'] = behaviors['article_ids_clicked'].apply(lambda x: 'N' + str(x) )
        behaviors['article_ids_clicked'] = behaviors['article_ids_clicked'].str.replace(' ', ' N').str.replace('[', '').str.replace(']', '')

    # Select the required columns
    df = behaviors[['user_id', 'impression_time', 'article_ids_clicked', 'article_ids_inview']]
    df['article_ids_inview'] = df['article_ids_inview'].str.replace('"', '')
    df['article_ids_inview'] = df.apply(process_ids, axis=1)

    if(not isinstance(history['impression_time_fixed'][0], str)):
        history['impression_time_fixed'] = history['impression_time_fixed'].apply(lambda x: x[-max_history:] if len(x)>max_history else x)
        history['article_id_fixed'] = history['article_id_fixed'].apply(lambda x: x[-max_history:] if len(x)>max_history else x)

    history['impression_time_fixed'] = list(history['impression_time_fixed'])
    # Check if 'impression_time' is already in the desired format
    if not history['impression_time_fixed'].str.contains('/').any():
        # Convert 'impression_time' to datetime
        history['impression_time_fixed'] = [pd.to_datetime(x.astype(str)) for x in history['impression_time_fixed']]

        # Format 'impression_time' in the desired format
        history['impression_time_fixed'] = [x.strftime('%m/%d/%Y %I:%M:%S %p') for x in history['impression_time_fixed']]
        # history['impression_time_fixed'] = list(history['impression_time_fixed'] )

    # Add 'U' before every 'user_id' if it doesn't already start with 'U'
    history['user_id'] = history['user_id'].apply(lambda x: 'U' + str(x) if not str(x).startswith('U') else str(x))

    # Same for article_ids_clicked
    if not history['article_id_fixed'].str.contains('N').any():
        history['article_id_fixed'] = history['article_id_fixed'].apply(lambda x: 'N' + str(x) )
        history['article_id_fixed'] = history['article_id_fixed'].str.replace(' ', ' N').str.replace('[', '').str.replace(']', '').str.replace('\n', ' ').str.replace('\t', ' ').str.replace('  ', ' ')
        # history['article_id_fixed'] = history['article_id_fixed'].str.split(' ')

    # Function to join times and user_ids
    def join_times_and_ids(row):
        # Zip the lists from the 'times' and 'user_ids' columns
        paired_list = zip(row['article_id_fixed'].split(' '), row['impression_time_fixed'])
        # Format each pair and join them into a list
        return [pair for pair in paired_list]

    # Apply the function to each row
    history['combined'] = history.apply(join_times_and_ids, axis=1)

    history = history[['user_id', 'combined']]

    

    # df['tot_clicks'] = df.apply(lambda row: (row['article_ids_clicked'], row['impression_time']),axis=1)
    df['positive'] = df['article_ids_clicked']
    df['negatives'] = df['article_ids_inview'].to_list()
    df['bucket'] = df['impression_time']

    # df_grouped = df.groupby('user_id').agg({'tot_clicks': lambda x: x.tolist()}).reset_index()

    df = df.merge(history[['user_id', 'combined']], on='user_id', how='left')

    docs = df[['user_id', 'combined', 'positive', 'negatives', 'bucket']]

    # Write the DataFrame back to the .tsv file
    # train_split = docs[:int(len(docs)*0.8)]
    val_split = docs[: int(len(docs)*0.2)]
    test_split = docs[int(len(docs)*0.2) : int(len(docs)*0.5)]

    if (trainset):
        docs.to_csv('../train.tsv', sep='\t', index=False, header=False)
    else:
        val_split.to_csv('../val.tsv', sep='\t', index=False, header=False)
        test_split.to_csv('../test.tsv', sep='\t', index=False, header=False)

    return docs

docs = write_data(behaviors, history, 100, True)
val_docs = write_data(val_behaviors, val_history, 100, False)





C:\Users\tangs\AppData\Local\Temp\ipykernel_12172\3813114995.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['article_ids_inview'] = df['article_ids_inview'].str.replace('"', '')
C:\Users\tangs\AppData\Local\Temp\ipykernel_12172\3813114995.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['article_ids_inview'] = df.apply(process_ids, axis=1)
C:\Users\tangs\AppData\Local\Temp\ipykernel_12172\3813114995.py:50: UserWarning: Could not infer format, so each element will be parsed individually, fall

In [64]:
history["article_id_fixed"][0]

'N9764070 N9764086 N9764049 N9761801 N9764325 N9758057 N9763854 N9764070 N9764361 N9764621 N9764822 N9764813 N9765244 N9764765 N9765175 N9764505 N9765067 N9765190 N9764988 N9764350 N9759717 N9765545 N9765551 N9765539 N9758572 N9765359 N9765410 N9765481 N9766419 N9766452 N9765641 N9766279 N9765803 N9766146 N9766348 N9766161 N9766129 N9758432 N9766015 N9761963 N9766099 N9766805 N9766108 N9754042 N9766627 N9754730 N9765846 N9766261 N9766242 N9766808 N9766862 N9766886 N9766944 N9762678 N9766695 N9765326 N9766592 N9766757 N9766770 N9768387 N9766599 N9766136 N9768129 N9768377 N9768311 N9766752 N9768150 N9768177 N9768087 N9768193 N9767342 N9767751 N9767908 N9768687 N9767557 N9767989 N9768599 N9768469 N9768860 N9768583 N9768829 N9762135 N9769994 N9770006 N9769891 N9769691 N9769909 N9769883 N9767852 N9769641 N9770328 N9769888 N9770030 N9769220 N9769994 N9768260 N9770594 N9770726 N9769404 N9769366'

In [65]:
val_docs['lens'] = val_docs['combined'].apply(len)

In [66]:
val_docs['lens'].min()

5

In [67]:
docs['combined'].sort_values()

2863     [(N4543921, 05/11/2023 11:39:44 AM), (N4543921...
17425    [(N4543921, 05/11/2023 11:39:44 AM), (N4543921...
20831    [(N4543921, 05/11/2023 11:39:44 AM), (N4543921...
5105     [(N4543921, 05/11/2023 11:39:44 AM), (N4543921...
10508    [(N4543921, 05/11/2023 11:39:44 AM), (N4543921...
                               ...                        
5196     [(N9770798, 05/17/2023 10:01:52 PM), (N9769306...
9785     [(N9770798, 05/17/2023 10:01:52 PM), (N9769306...
12625    [(N9770829, 05/18/2023 05:08:34 AM), (N9769553...
9990     [(N9770829, 05/18/2023 05:08:34 AM), (N9769553...
22569    [(N9770829, 05/18/2023 05:08:34 AM), (N9769553...
Name: combined, Length: 24724, dtype: object

## Create docs.tsv

In [68]:

# Select the required columns
articles['body'] = articles['body'].str.replace('\n', ' ')
articles['title'] = articles['title'].str.replace('\n', ' ')
articles['body'] = articles['body'].str.replace('\t', ' ')
# articles['body'] = articles['body'].fillna('')
# articles['category_str'] = articles['category_str'].fillna('')
articles['topics'] = articles['topics'].astype(str).str.replace('\n', ' ')
articles['article_id'] = articles['article_id'].apply(lambda x: 'N' + str(x) if not str(x).startswith('N') else str(x))
docs_file = articles[['article_id', 'title', 'category_str', 'subcategory', 'body', 'topics']]

# Write to a new .tsv file without index
docs_file.to_csv('../docs.tsv', sep='\t', index=False, header=False)

## Create docs_pub_time.tsv

In [69]:
articles['article_id'] = articles['article_id'].apply(lambda x: 'N' + str(x) if not str(x).startswith('N') else str(x))

# Convert 'impression_time' to string
articles['published_time'] = articles['published_time'].astype(str)

# Check if 'impression_time' is already in the desired format
if not articles['published_time'].str.contains('/').any():
    # Convert 'impression_time' to datetime
    articles['published_time'] = pd.to_datetime(articles['published_time'])

    # Format 'impression_time' in the desired format
    articles['published_time'] = articles['published_time'].dt.strftime('%m/%d/%Y %I:%M:%S %p')
pubdocs_file = articles[['article_id', 'published_time']]

# Write to a new .tsv file without index
pubdocs_file.to_csv('../popularity/docs_pub_time.tsv', sep='\t', index=False, header=False)

## Create entities